References
- [Dicoding - Mendapatkan Prediksi Model dari Input Text](https://www.dicoding.com/academies/443/discussions/209385)

In [50]:
import tensorflow as tf
import pandas as pd
import requests
import json
import base64
import random

In [51]:
df = pd.read_csv('data/healthcare-dataset-stroke-data.csv')
df.pop('stroke')

columns = df.columns.values
random = random.randint(0, len(columns))

features = df.values[random]

inputs = {key: value for key, value in zip(columns, features)}
inputs

{'gender': 'Female',
 'age': 69,
 'hypertension': 0,
 'heart_disease': 0,
 'ever_married': 'No',
 'work_type': 'Private',
 'Residence_type': 'Urban',
 'avg_glucose_level': 94.39,
 'bmi': 22.8,
 'smoking_status': 'never smoked'}

In [52]:
def int_feature(value):
    return tf.train.Feature(
        int64_list=tf.train.Int64List(
            value=[value]
        ),
    )

def float_feature(value):
    return tf.train.Feature(
        float_list=tf.train.FloatList(
            value=[value]
        ),
    )

def string_feature(value):
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(
            value=[bytes(value, 'utf-8')]
        ),
    )

In [53]:
def prepare_json(inputs: dict):
    feature_spec = dict()

    for keys, values in inputs.items():
        if isinstance(values, int):
            feature_spec[keys] = int_feature(values)
        elif isinstance(values, float):
            feature_spec[keys] = float_feature(values)
        elif isinstance(values, str):
            feature_spec[keys] = string_feature(values)

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [{
        'examples': {
            'b64': base64.b64encode(example).decode()
        }
    }]

    return json.dumps({
        'signature_name': 'serving_default',
        'instances': result,
    })

In [54]:
def predict(inputs):
    response = requests.post(
        'https://stroke-detection.up.railway.app/v1/models/stroke-detection-model:predict',
        data=prepare_json(inputs)
    )

    predict = response.json()['predictions'][0][0]
    return predict

In [55]:
predict = predict(inputs)

print(f'{inputs}\n')
print(f'Stroke' if predict > 0.6 else f'Not Stroke')

{'gender': 'Female', 'age': 69, 'hypertension': 0, 'heart_disease': 0, 'ever_married': 'No', 'work_type': 'Private', 'Residence_type': 'Urban', 'avg_glucose_level': 94.39, 'bmi': 22.8, 'smoking_status': 'never smoked'}

Not Stroke
